In [24]:
with open("../backend/logging/search.log", "r") as f, open("pre_file.csv", "w") as f2:
    for line_ in f.readlines():

        time, event, query, documents_ = line_.split(";")
        time, event, query, documents_ = time, event.strip(), query.strip(), documents_.strip()
        if event == "Search.viewed":
            documents = documents_.split("|")

            for document in documents:
                f2.write(f"{query},{document},1,0\n")
        else:
            f2.write(f"{query},{documents_},0,1\n")

In [25]:
!sort -t ',' -k 1n pre_file.csv > pre_file_sorted.csv

In [26]:
!rm pre_file.csv

In [27]:
now_q = ""
now_d = ""

v_c = (0,0)

first = True

with open("pre_file_sorted.csv", "r") as f3, open("features_file.csv", "w") as f4:
    for line_ in f3.readlines():
        query, document, view, click = [x.strip() for x in line_.split(",")]

        if first:
            now_q = query
            now_d = document
            v_c = (int(view),int(click))
            first = False

        if (now_q, now_d) != (query,document):
            f4.write(f"{now_q},{now_d},{v_c[0]},{v_c[1]}\n")
            now_q = query
            now_d = document
            v_c = (int(view),int(click))
        else:
            v_c = (v_c[0] + int(view), v_c[1] + int(click))